In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install datasets
!pip install --upgrade torch transformers
!pip install accelerate peft
!pip install pymupdf
!npm install localtunnel
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com
!npm install localtunnel
!streamlit run app.py & npx localtunnel --port 8501


In [ ]:
my_path = "VarunResume.docx"
doc = fitz.open(my_path)

In [ ]:
text1 = ""
for page in doc:
    text = page.get_text()
    output = page.get_text("blocks")
    output
    for block in output:
        text1 += block[4]

In [ ]:
hf_token = "HF_API_KEY"
os.environ['HF_TOKEN'] = hf_token

In [ ]:
required_skills = "tableau, aws, power bi, flask, angular"

In [ ]:
import pandas as pd
input_values = "Python 2, Oracle 5, angular 9, java 10"

values_list = input_values.split(", ")

required_skills = []
required_experiences = []

# Iterate through the values list and extract skill and experience
for value in values_list:
    skill, experience = value.split(" ")
    required_skills.append(skill.title())  # Ensure title case for skill
    required_experiences.append(int(experience))

# Create a DataFrame
df_required_skill = pd.DataFrame({
    'skill': required_skills,
    'required_experience': required_experiences
})


In [ ]:
df_skills = df_required_skill.merge(df_grouped, on='skill', how='left')
df_skills['experience'].fillna(0, inplace=True)
json_data = []
for index, row in df_skills.iterrows():
    skill_info = {
        row['skill']: {
            'required_experience': row['required_experience'],
            'experience': row['experience']
        }
    }
    json_data.append(skill_info)

# Print the JSON data
print(json_data)

In [ ]:
json_data = "[{'Python': {'required_experience': 2, 'experience': 0.0}}, {'Oracle': {'required_experience': 5, 'experience': 7.2}}, {'Angular': {'required_experience': 9, 'experience': 0.0}}, {'Java': {'required_experience': 10, 'experience': 7.2}}]"


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import plotly.express as px
import fitz
import torch
import os
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from transformers import AutoTokenizer, TextStreamer, pipeline,LlamaForCausalLM,AutoModelForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import LLMChain
import plotly.express as px
from accelerate import Accelerator
import numpy as np



def load_model():
  model_name = "mistralai/Mistral-7B-Instruct-v0.2"
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      torch_dtype=torch.float16,
      # load_in_8bit=True,
      # load_in_4bit=True,
      device_map="auto",
      use_cache=True,
  )
  DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
  streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

  text_pipeline = pipeline(
      "text-generation",
      model=model,
      tokenizer=tokenizer,
      max_new_tokens=5000,
      do_sample=False,
      repetition_penalty=1.15,
      streamer=streamer
  )

  llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.1})
  accelerator = Accelerator()

  config = {'max_new_tokens': 512, 'repetition_penalty': 1.1, 'context_length': 8000, 'temperature':0, 'gpu_layers':50}
  llm, config = accelerator.prepare(llm, config)
  return llm

def get_person_details(text, llm):
  person_prompt_tpl1 = """From the Resume text for a job aspirant below, extract Entities strictly as instructed below:

  1. First, look for the Person Entity type in the text and extract the needed information defined below:
    `id` property of each entity must be alphanumeric and must be unique among the entities. You will be referring this property to define the relationship between entities.
      Entity Types:
      label:'Person',id:string,name:string,email:string,phone:string //Person Node

  2. If you cannot find any information on the entities & relationships above, it is okay to return empty value. DO NOT create fictitious data.
  3. Do NOT create duplicate entities.
  4. Restrict yourself to extract only Name, Email, and Phone number information. No other details should be focused.
  5. NEVER Impute missing values.

  Output JSON:
  {{"entities":{{"Person Details":[{{"name":"John Doe","email":"johndoe@example.com","phone":"123-456-7890"}}]}}}}

  Question: Now, extract the Person for the text below -

  {text}

  Answer:
  """
  prompttemplate=PromptTemplate(template=person_prompt_tpl1,input_variables=['text'])
  chain=LLMChain(llm=llm,prompt=prompttemplate)
  result=chain(text1)

  start_word = 'Answer'
  start_index = result['text'].index(start_word)
  sliced_text1 = result['text'][start_index:]

  sliced_text1 = sliced_text1.replace('Answer:\n', '')
  json_data1 = json.loads(sliced_text1)
  json1 = [json_data1['entities']]

  return json1

def get_education(text, llm):
  edu_prompt_tpl="""From the Resume text for a job aspirant below, extract Entities strictly as instructed below
  1. Look for Education entity type and generate the information defined below:
    `id` property of each entity must be alphanumeric and must be unique among the entities. You will be referring this property to define the relationship between entities. NEVER create other entity types that aren't mentioned below. You will have to generate as many entities as needed as per the types below:
      Entity Definition:
      label:'Education',id:string,degree:string,university:string,graduationDate:string,score:string,url:string //Education Node
  2. If you cannot find any information on the entities above, it is okay to return empty value. DO NOT create fictious data
  3. Do NOT create duplicate entities or properties
  4. Strictly extract only Education. No Skill or other Entities should be extracted
  5. DO NOT MISS out any Education related entity
  6. NEVER Impute missing values
  7. Always look for GPA and return it in the score section
  7. Extract only Degree University, Graduation Date, GPA/Score and nothing else

  Output JSON (Strict):
  {{"entities":{{"Education":[{{"degree":"Bachelor of Science","graduationDate":"May 2022","score":"0.0"}}]}}}}


  Question: Now, extract Education information as mentioned above for the text below -

  {text}

  Answer:
  """

  prompttemplate=PromptTemplate(template=edu_prompt_tpl,input_variables=['text'])
  chain=LLMChain(llm=llm,prompt=prompttemplate)
  result=chain(text1)

  start_word = 'Answer'
  start_index = result['text'].index(start_word)
  sliced_text2 = result['text'][start_index:]

  sliced_text2 = sliced_text2.replace('Answer:\n', '')
  json_data2 = json.loads(sliced_text2)
  json2 = [json_data2['entities']]

  return json2


def get_skills(text, llm):
  skill_prompt_tpl1 = """From the Resume text below, extract Entities strictly as instructed below:

  1. Look for Experience Entities in the text. The `id` property of each entity must be alphanumeric and must be unique among the entities. NEVER create new entity types that aren't mentioned below:
      Entity Definition:
      label:'Skills',name:string,category:string,years:string //Skills Node

  2. Extract 5 skills from each of the experiences in the text. If the skill is currently held by the individual, calculate the total experience until the current date and append "Current" beside it.

  3. Calculate the total experience in years and add it as the "years" property.

  4. NEVER Impute missing values.

  Output JSON (Strict):
  {{"entities":{{"Skills":[{{"name":"Python","category":"Programming", "years":"2.3"}},{{"name":"SQL","category":"Programming", "years":"0.4"}}]}}}}


  Question: Now, extract entities as mentioned above for the text below -

  {text}

  Answer:
  """

  prompttemplate=PromptTemplate(template=skill_prompt_tpl1,input_variables=['text'])
  chain=LLMChain(llm=llm,prompt=prompttemplate)
  result=chain(text1)

  start_word = 'Answer'
  start_index = result['text'].index(start_word)
  sliced_text3 = result['text'][start_index:]

  sliced_text3 = sliced_text3.replace('Answer:\n', '')
  json_data3 = json.loads(sliced_text3)
  json3 = [json_data3['entities']]

  return json3

def get_experience(text, llm):
  exp_prompt_tpl = """From the Resume text below, extract Entities strictly as instructed below:

  1. Look for Experience Entities in the text. The `id` property of each entity must be alphanumeric and must be unique among the entities. NEVER create new entity types that aren't mentioned below:
      Entity Definition:
      label:'Experience',id:string,job_title:string,company:string,years:string //Experience Node

  2. NEVER Impute missing values.

  3. If the job is currently held by the individual, calculate the total experience until the current date and append "Current" beside it.

  4. Calculate the total experience in either years or months (if experience is less than a year) and add it as the "years" property.

  5. Don't write any additional notes at the end.

  Example Output Format:
  {{"entities": [{{"label":"Experience","id":"exp1","job_title":"Software Engineer","company":"ABC Technologies","years":"3"}},{{"label":"Experience","id":"exp2","job_title":"Data Analyst","company":"XYZ Inc.","years":"5"}}]}}

  Question: Now, extract entities as mentioned above for the text below -
  {text}

  Answer:

  """
  prompttemplate=PromptTemplate(template=exp_prompt_tpl,input_variables=['text'])
  chain=LLMChain(llm=llm,prompt=prompttemplate)
  result=chain(text1)

  start_word = 'Answer'
  start_index = result['text'].index(start_word)
  sliced_text4 = result['text'][start_index:]

  sliced_text4 = sliced_text4.replace('Answer:\n', '')
  json_data4 = json.loads(sliced_text4)
  json4 = json_data4['entities']

  return json4


def combine_jsons(j1,j2,j3,j4):
  combined_json = {}
  for dictionary in j1:
    combined_json.update(dictionary)
  for dictionary in j2:
    combined_json.update(dictionary)
  for dictionary in j3:
    combined_json.update(dictionary)
  for dictionary in j4:
    combined_json.update(dictionary)

  return combined_json


def get_resume_entities(text, llm):
  j1 = get_person_details(text, llm)
  j2 = get_education(text, llm)
  j3 = get_skills(text, llm)
  j4 = get_experience(text, llm)
  combined_json = combine_jsons(j1,j2,j3,j4)
  return combined_json
st.set_page_config(page_title='Resume Analysis', layout='wide')
css = """
.stApp > header {
        color: #fff;
        background-color: white;
  }

.stApp {
    margin: auto;
    overflow: auto;
    animation: gradient 15s ease infinite;
    background-size: 400% 400%;
    background-attachment: fixed;
   background-color: #0FA4AF;
}
    .stFileUploader {
        width: max-content;
    }
    .stFileUploader section {
        padding: 0;
        float: left;
    }
    .stFileUploader section > input + div {
        display: none;
    }
    .stFileUploader section + div {
        float: right;
        padding-top: 0;
    }
  .logo-container {
        display: flex;
        justify-content: center;
        align-items: center; /* Center-align vertically */
        height: 100vh; /* Set height to 100% of viewport height */
    }
    .logo-image {
        width: 300px; /* Adjust the width of the logo */
        height: auto;
    }
"""
with st.columns(3)[1]:
  st.markdown("<h2 style='text-align: center; color: black;'>Resume Analysis</h2>", unsafe_allow_html=True)
st.markdown(f'<style>{css}</style>', unsafe_allow_html=True)
# Display logo image with custom CSS styles
with st.columns(3)[1]:
     st.image('logo.png', width=300, output_format='auto')

url = 'https://fonts.googleapis.com/icon?family=Material+Icons'
st.markdown(f'<link href="{url}" rel="stylesheet">', unsafe_allow_html=True)

uploaded_file = st.sidebar.file_uploader(
    label="Upload Resume",
    accept_multiple_files = False
)

if not uploaded_file:
  st.info('Please Upload Resume to continue')
  st.stop()

resume_text = uploaded_file.read()
doc = fitz.open(stream=resume_text, filetype="pdf")

# Initialize an empty string to store the text
text1 = ''

# Extract text from each page using "blocks" and concatenate it
for page in doc:
    blocks = page.get_text("blocks")
    for block in blocks:
        text1 += block[4]


llm = load_model()
#result_details = get_resume_entities(text1, llm)
skill_details = get_skills(text1, llm)
st.write('Please wait.. Loading..')
skills_df = pd.DataFrame(skill_details[0]['Skills'])
skills_df = skills_df.sort_values(by='years', ascending=False).head(10)
skills_df['Required Experience'] = np.random.randint(1, 6, size=len(skills_df))
skills_df.rename(columns={'years':'Experience'}, inplace=True)
skills_df['Required Experience'] = skills_df['Required Experience'].astype('float')
skills_df['Experience'] = skills_df['Experience'].astype('float')
colors = {'Experience': 'black', 'Required Experience': 'red'}
fig = px.bar(skills_df,
             x=['Experience', 'Required Experience'],
             y='name',
             barmode='group',
             labels={'value': 'Years / Required Experience', 'name': 'Skills'},
             title='Skills Comparison',
             color_discrete_map=colors,
             )

fig.update_layout(xaxis_title='Skill', yaxis_title='Experience')
fig.update_layout(
        font=dict(
            family='Arial, sans-serif',
            size=14,
            color='black'
        ),
        template='plotly_white',
        legend=dict(orientation="h", y=-0.15)
    )
fig.update_layout(
  legend_title='Category',
)
fig.update_layout(width=900, height=900)

st.plotly_chart(fig)

In [ ]:
%%writefile app.py
css = """
.stApp > header {
        color: #fff;
        background-color: red;
  }

.stApp {
    margin: auto;
    overflow: auto;
    animation: gradient 15s ease infinite;
    background-size: 400% 400%;
    background-attachment: fixed;
   background-color: black;
}
  .logo-container {
        display: flex;
        justify-content: center;
        align-items: center; /* Center-align vertically */
        height: 100vh; /* Set height to 100% of viewport height */
    }
    .logo-image {
        width: 300px; /* Adjust the width of the logo */
        height: auto;
    }
"""
import streamlit as st
st.markdown(f'<style>{css}</style>', unsafe_allow_html=True)

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import plotly.express as px
import fitz
import torch
import os
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from transformers import AutoTokenizer, TextStreamer, pipeline,LlamaForCausalLM,AutoModelForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import LLMChain
import plotly.express as px
from accelerate import Accelerator
import numpy as np



def load_model():
  model_name = "mistralai/Mistral-7B-Instruct-v0.2"
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      torch_dtype=torch.float16,
      # load_in_8bit=True,
      # load_in_4bit=True,
      device_map="auto",
      use_cache=True,
  )
  DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
  streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

  text_pipeline = pipeline(
      "text-generation",
      model=model,
      tokenizer=tokenizer,
      max_new_tokens=5000,
      do_sample=False,
      repetition_penalty=1.15,
      streamer=streamer
  )

  llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.1})
  accelerator = Accelerator()

  config = {'max_new_tokens': 512, 'repetition_penalty': 1.1, 'context_length': 8000, 'temperature':0, 'gpu_layers':50}
  llm, config = accelerator.prepare(llm, config)
  return llm

def get_skills(text, llm):
  skill_prompt_tpl1 = """From the Resume text below, extract Entities strictly as instructed below:

  1. Look for Experience Entities in the text.
      Entity Definition:
      label:'Skills',name:string,category:string,years:string //Skills Node

  2. Get Top 10 technical skills from each professional experience.

  3. Do NOT create duplicate entities.

  4. Have just one technical skills section for each professional experience.

  5. NEVER Impute missing values.

  6/ Calculate duration as end_date - start_date of professional experience. For example May 2022 - June 2021 = 1.2.

  Output JSON (Strict) as an example:
  {{"entities":{{"Name of company":[{{ "skills": ["Java", "SQL", "c","d","e","f","g","i","h","k"]}}, {{"duration":"2.9"}}]}}}}

  Question: Now, extract entities as mentioned above for the text below -

  {text}

  Answer:
  """

  prompttemplate=PromptTemplate(template=skill_prompt_tpl1,input_variables=['text'])
  chain=LLMChain(llm=llm,prompt=prompttemplate)
  result=chain(text1)

  start_word = 'Answer'
  start_index = result['text'].index(start_word)
  sliced_text3 = result['text'][start_index:]

  sliced_text3 = sliced_text3.replace('Answer:\n', '')
  json_data3 = json.loads(sliced_text3)
  json3 = [json_data3['entities']]

  return json3


st.set_page_config(page_title='ResumeScan', layout='wide')

css = """
.stApp > header {
        color: #fff;
        background-color: white;
  }

.stApp {
    margin: auto;
    overflow: auto;
    animation: gradient 15s ease infinite;
    background-size: 400% 400%;
    background-attachment: fixed;
   background-color: #0FA4AF;
}
    .stFileUploader {
        width: max-content;
    }
    .stFileUploader section {
        padding: 0;
        float: left;
    }
    .stFileUploader section > input + div {
        display: none;
    }
    .stFileUploader section + div {
        float: right;
        padding-top: 0;
    }
  .logo-container {
        display: flex;
        justify-content: center;
        align-items: center; /* Center-align vertically */
        height: 100vh; /* Set height to 100% of viewport height */
    }
    .logo-image {
        width: 300px; /* Adjust the width of the logo */
        height: auto;
    }
"""

with st.columns(3)[1]:
  st.markdown("<h2 style='text-align: center; color: black;'>Resume Analysis</h2>", unsafe_allow_html=True)
  st.markdown(f'<style>{css}</style>', unsafe_allow_html=True)
# Display logo image with custom CSS styles
with st.columns(3)[1]:
     st.image('logo.png', width=300, output_format='auto')

url = 'https://fonts.googleapis.com/icon?family=Material+Icons'
st.markdown(f'<link href="{url}" rel="stylesheet">', unsafe_allow_html=True)

uploaded_file = st.sidebar.file_uploader(
    label="Upload Resume",
    accept_multiple_files = False
)

if not uploaded_file:
  st.info('Please Upload Resume to continue')
  st.stop()

resume_text = uploaded_file.read()
doc = fitz.open(stream=resume_text)

text1 = ""
for page in doc:
    text = page.get_text()
    output = page.get_text("blocks")
    for block in output:
        text1 += block[4]

llm = load_model()
st.write('Model Loaded...')
#result_details = get_resume_entities(text1, llm)
skill_details = get_skills(text1, llm)

st.write('Please wait.. Loading..')
st.write(skill_details)
company_list = []
skills_list = []
duration_list = []
for company_data in skill_details:
    for company_name, skills_data in company_data.items():
        for skills_info in skills_data:
            skills = skills_info['skills']
            if 'duration' in skills_info:
              duration = skills_info['duration']
            skills_list.extend(skills)
            duration_list.extend([duration] * len(skills))
            company_list.extend([company_name] * len(skills))

df = pd.DataFrame({'company': company_list, 'skill': skills_list, 'duration': duration_list})

skills_df = df.sort_values(by='duration', ascending=False).head(10)
skills_df.rename(columns={'duration':'Experience'}, inplace=True)

skills_df['Required Experience'] = np.random.randint(1, 6, size=len(skills_df))
skills_df.rename(columns={'years':'Experience'}, inplace=True)
skills_df['Required Experience'] = skills_df['Required Experience'].astype('float')
skills_df['Experience'] = skills_df['Experience'].astype('float')
colors = {'Experience': 'black', 'Required Experience': 'red'}

fig = px.bar(skills_df,
             x=['Experience', 'Required Experience'],
             y='skill',
             barmode='group',
             labels={'value': 'Years / Required Experience', 'skill': 'Skills'},
             color_discrete_map=colors,
             )

st.title('Skills Comparision')
fig.update_layout(xaxis_title='Skill', yaxis_title='Experience')
fig.update_layout(
        font=dict(
            family='Arial, sans-serif',
            size=14,
            color='black'
        ),
        template='plotly_white',
        legend=dict(orientation="h", y=-0.15)
    )
fig.update_layout(
  legend_title='Category',
)
fig.update_layout(width=800, height=600)

st.plotly_chart(fig)